In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
plt.close('all')

/Users/khw/.pyenv/versions/3.9.1/envs/assembly/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
frames = []
import traceback
for txt in os.listdir('./aol')[0:1]:
    try:
        queries = pd.read_csv(f'./aol/{txt}', sep='\t', dtype={"AnonID": "Int64", "Query": "string", "QueryTime": "string", "ItemRank": "Int32", "ClickURL": "string"})
        frames.append(queries)
    except Exception as e:
        print(txt)
        print(e)
        print(e.args)
        traceback.print_exc()

queries = pd.concat(frames)
queries['QueryTime'] = pd.to_datetime(queries['QueryTime'])

In [5]:
print(queries[0:30])

    AnonID                        Query           QueryTime  ItemRank  \
0      142               rentdirect.com 2006-03-01 07:17:12      <NA>   
1      142  www.prescriptionfortime.com 2006-03-12 12:31:06      <NA>   
2      142                   staple.com 2006-03-17 21:19:29      <NA>   
3      142                   staple.com 2006-03-17 21:19:45      <NA>   
4      142    www.newyorklawyersite.com 2006-03-18 08:02:58      <NA>   
5      142    www.newyorklawyersite.com 2006-03-18 08:03:09      <NA>   
6      142              westchester.gov 2006-03-20 03:55:57         1   
7      142                space.comhttp 2006-03-24 20:51:24      <NA>   
8      142                         dfdf 2006-03-24 22:23:07      <NA>   
9      142                         dfdf 2006-03-24 22:23:14      <NA>   
10     142                  vaniqa.comh 2006-03-25 23:27:12      <NA>   
11     142          www.collegeucla.edu 2006-04-03 21:12:14      <NA>   
12     142                   www.elaorg 2006-04-03 

In [3]:
unique_ids = pd.unique(queries['AnonID'])

print(unique_ids)

grouped_by_id = queries.groupby('AnonID')

<IntegerArray>
[     142,      217,      993,     1268,     1326,     1337,     1410,
     2005,     2178,     2334,
 ...
 24965344, 24965559, 24967152, 24967317, 24967503, 24967641, 24967786,
 24968114, 24969251, 24969374]
Length: 65516, dtype: Int64


In [26]:
qqq = grouped_by_id['Query'].nunique()
ppp = qqq.sort_values(ascending=False)

filtered = ppp.index


In [27]:
queries.insert(5, 'Gap', 0)
queries.insert(6, 'SessionNum', 0)
gaps = pd.Series(dtype='timedelta64[ns]')

sessions = []

ValueError: cannot insert Gap, already exists

In [28]:
for id in filtered:
    queries.loc[queries['AnonID'] == id, 'Gap'] = queries.loc[queries['AnonID'] == id, 'QueryTime'].diff()
    queries.loc[queries['AnonID'] == id, 'SessionNum'] = queries.loc[queries['AnonID'] == id, 'Gap'] > pd.Timedelta(minutes = 30)
    queries.loc[queries['AnonID'] == id, 'SessionNum'] = queries.loc[queries['AnonID'] == id, 'SessionNum'].apply(lambda x: 1 if x else 0).cumsum()
    
    all_sessions = queries[queries['AnonID'] == id]
    grouped = all_sessions.groupby('SessionNum')         
    gaps = pd.concat([gaps, all_sessions['Gap']])


KeyboardInterrupt: 